In [ ]:
## Install the required libraries

# !pip install biopython
# !pip install reportlab


from Bio import SeqIO

from Bio.SeqFeature import SeqFeature, FeatureLocation

In [ ]:
## reading the genbank file

gb_file = "Genome.gb"
for gb_record in SeqIO.parse(open(gb_file,"r"), "genbank"):
    print(f"Total Features: {len(gb_record.features)}")

## Exploratry Analysis

In [ ]:
# print last feature
print(gb_record.features[10])

I am going to draw a whole genome from a SeqRecord object read in from a GenBank file

In [ ]:
# importing necessary libraries for
# Visualization and graphs
from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram
from Bio import SeqIO
# reading the file again for graph
record = SeqIO.read("Genome.gb", "genbank")

## Top down approach

In [ ]:
## I am using a top down approach so after loading in our sequence we next create an empty diagram, 
## then add an (empty) track, and to that add an (empty) feature set
gd_diagram = GenomeDiagram.Diagram("Genome analysis")
gd_track_for_features = gd_diagram.new_track(1, name="Annotated Features")
gd_feature_set = gd_track_for_features.new_set()

In [ ]:
## I take each gene SeqFeature object in our SeqRecord, and use it to generate a feature on the diagram. 
## I am going to color them blue, alternating between a dark blue and a light blue.

for feature in record.features:
    if feature.type != "gene":
        #Exclude this feature
        continue
    if len(gd_feature_set) % 2 == 0:
        color = colors.blue
    else:
        color = colors.lightblue
    gd_feature_set.add_feature(feature, color=color, label=True)

In [ ]:
## drawing a circular genome map and output it as a PNG file

## Now i come to actually making the output file. 
## This happens in two steps, first we call the draw method, which creates all the shapes using ReportLab objects. 

## Then we call the write method which renders these to the requested file format. 

gd_diagram.draw(format="circular", circular=True, pagesize=(20*cm,20*cm),
                start=0, end=len(record), circle_core=0.7)
gd_diagram.write("genome circular.png", "PNG")
# saved in the folder

## Bonus

In [ ]:
## This time we’ll use arrows for the genes, 
#  and overlay them with strand-less features (as plain boxes) showing the position of some restriction digest sites.

for feature in record.features:
    if feature.type != "gene":
        #Exclude this feature
        continue
    if len(gd_feature_set) % 2 == 0:
        color = colors.blue
    else:
        color = colors.lightblue
    gd_feature_set.add_feature(feature, sigil="ARROW",
                               color=color, label=True,
                               label_size = 14, label_angle=0)

In [ ]:
#I want to include some strandless features, so for an example
#will use EcoRI recognition sites etc.

for site, name, color in [("GAATTC","EcoRI",colors.green),
                          ("CCCGGG","SmaI",colors.orange),
                          ("AAGCTT","HindIII",colors.red),
                          ("GGATCC","BamHI",colors.purple)]:
    index = 0
    while True:
        index  = record.seq.find(site, start=index)
        if index == -1 : break
        feature = SeqFeature(FeatureLocation(index, index+len(site)))
        gd_feature_set.add_feature(feature, color=color, name=name,
                                   label=True, label_size = 10,
                                   label_color=color)
        index += len(site)

In [ ]:
gd_diagram.draw(format="circular", circular=True, pagesize=(20*cm,20*cm),
                start=0, end=len(record), circle_core = 0.5)
gd_diagram.write("genome circular nice.png", "PNG")